In [1]:
!pip install tensorflow
!pip install tensorflow mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 61.0 MB/s eta 0:00:00


In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import pickle
from tqdm import tqdm
import cv2
from concurrent.futures import ThreadPoolExecutor

# GPU setup
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=15000)]  # Adjust as needed
            )
            print(f"Configured GPU with a memory limit of 15000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

# Mixed precision setup
try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

# Save and load progress functions
def save_progress(filepath, data):
    try:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        print(f"Progress saved at: {filepath}")
    except Exception as e:
        print(f"Error saving progress: {e}")

def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading progress: {e}")
    return []

# Image preprocessing
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return np.zeros((224, 224, 3), dtype=np.float32)

        image = cv2.resize(image, (224, 224))
        image = preprocess_input(image.astype(np.float32))
        return image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return np.zeros((224, 224, 3), dtype=np.float32)

# Multithreaded preprocessing
def preprocess_images_multithreaded(image_files, num_threads=4):
    processed_images = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(preprocess_image, image_files), total=len(image_files), desc="Preprocessing images"))
        processed_images.extend(results)
    return np.array(processed_images, dtype=np.float32)

# Spatial feature extraction for individual images
def extract_spatial_features_for_images(image_dir, model, checkpoint_path, batch_size=32, save_interval=200, num_threads=4):
    features = load_progress(checkpoint_path) or []
    processed_images = {f['image_name'] for f in features if 'image_name' in f}
    all_images = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    remaining_images = [img for img in all_images if img not in processed_images]

    print(f"Total images: {len(all_images)}, Remaining images: {len(remaining_images)}")

    for i in tqdm(range(0, len(remaining_images), batch_size), desc="Extracting spatial features"):
        batch_files = remaining_images[i:i+batch_size]

        try:
            # Multithreaded preprocessing
            processed_batch = preprocess_images_multithreaded(batch_files, num_threads)
            dataset = tf.data.Dataset.from_tensor_slices(processed_batch).batch(batch_size).prefetch(tf.data.AUTOTUNE)

            features_batch = model.predict(dataset, verbose=1)

            for img_path, feature in zip(batch_files, features_batch):
                features.append({'image_name': img_path, 'features': feature})
        except Exception as e:
            print(f"Error extracting features for batch {i // batch_size + 1}: {e}")

        # Save progress periodically
        if len(features) % save_interval == 0:
            save_progress(checkpoint_path, features)

    save_progress(checkpoint_path, features)
    return features

if __name__ == "__main__":
    # Define paths
    base_path = 'drive/MyDrive/SP_cup/fake/'
    features_dir = 'drive/MyDrive/SP_cup/features/fake'
    os.makedirs(features_dir, exist_ok=True)

    real_images_dir = os.path.join(base_path, 'fake-17')
    real_checkpoint = os.path.join(features_dir, 'spatial_features_fake_images-16.pkl')

    # Load EfficientNet model
    efficientnet = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

    # Extract spatial features for real images
    print("Processing real images...")
    spatial_features_real = extract_spatial_features_for_images(
        image_dir=real_images_dir,
        model=efficientnet,
        checkpoint_path=real_checkpoint,
        batch_size=32,
        save_interval=200,
        num_threads=4
    )

    print("Spatial feature extraction for images completed.")

Error configuring GPU: Cannot set memory growth on device when virtual devices configured
Mixed precision enabled for speedup.
Processing real images...
Total images: 4714, Remaining images: 4714


Preprocessing images: 100%|██████████| 32/32 [00:05<00:00,  6.37it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Preprocessing images: 100%|██████████| 32/32 [00:05<00:00,  6.18it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


Preprocessing images: 100%|██████████| 10/10 [00:00<00:00, 203.42it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


Extracting spatial features: 100%|██████████| 148/148 [02:19<00:00,  1.06it/s]

Progress saved at: drive/MyDrive/SP_cup/features/fake/spatial_features_fake_images-16.pkl
Spatial feature extraction for images completed.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
